In [0]:
# Databricks notebook: reset_registry_for_batch (simplified)
from datetime import datetime
import json
from pyspark.sql import functions as F
from delta.tables import DeltaTable

FILE_REG_TABLE = "census.bronze.file_registry_v1"

# Get batch ID from widget or use the latest
try:
    batch_id = dbutils.widgets.get("ingestion_batch_id")
    if batch_id is None or batch_id.strip() == "":
        # Get the latest batch with attempts
        fr = spark.table(FILE_REG_TABLE)
        latest = fr.filter(F.col("ingestion_attempts") > 0) \
            .select("ingestion_batch_id") \
            .distinct() \
            .orderBy(F.desc("ingestion_batch_id")) \
            .first()
        batch_id = latest["ingestion_batch_id"] if latest else None
except Exception:
    batch_id = None

if not batch_id:
    dbutils.notebook.exit(json.dumps({
        "status": "ERROR",
        "reason": "No batch ID provided or found"
    }))

# Reset Failed/Processing rows to Pending
dt = DeltaTable.forName(spark, FILE_REG_TABLE)

try:
    # Count before update
    count_before = spark.table(FILE_REG_TABLE) \
        .filter((F.col("ingestion_batch_id") == batch_id) & 
                (F.col("ingestion_status").isin(["Failed", "Processing"]))) \
        .count()
    
    if count_before == 0:
        dbutils.notebook.exit(json.dumps({
            "status": "OK",
            "ingestion_batch_id": batch_id,
            "note": "No Failed/Processing rows found to reset",
            "rows_updated": 0
        }))
    
    # Perform the update
    dt.update(
        condition=(F.col("ingestion_batch_id") == batch_id) & 
                 (F.col("ingestion_status").isin(["Failed", "Processing"])),
        set={
            "ingestion_status": F.lit("Pending"),
            "updated_at": F.lit(datetime.utcnow())
        }
    )
    
    dbutils.notebook.exit(json.dumps({
        "status": "OK",
        "ingestion_batch_id": batch_id,
        "note": f"Reset {count_before} rows to Pending status",
        "rows_updated": count_before
    }))
    
except Exception as e:
    # Simple error handling
    dbutils.notebook.exit(json.dumps({
        "status": "ERROR",
        "ingestion_batch_id": batch_id,
        "error": f"Failed to reset rows: {str(e)}"
    }))